In [ ]:
! pip install -U botocore==1.10.84 boto3==1.7.4 sagemaker==1.12.0

In [1]:
import os
import json
import boto3
import sagemaker

- Get the below three from your AWS account.
- You can also assign your ec2 instances IAM roles ans use `aws configure` to connect without explicitly providing these keys

In [2]:
AWS_ACCESS_KEY = os.environ.get('AWS_ACCESS_KEY')
AWS_SECRET_KEY = os.environ.get('AWS_SECRET_KEY')
AWS_REGION = os.environ.get('AWS_REGION', 'us-east-2')

In [3]:
boto_session = boto3.session.Session(aws_access_key_id=AWS_ACCESS_KEY, aws_secret_access_key=AWS_SECRET_KEY, region_name=AWS_REGION)
sagemaker_session = sagemaker.session.Session(boto_session=boto_session)

# If you have changed the enpoint in your account, please change it below
real_time_predictor = sagemaker.predictor.RealTimePredictor(endpoint='short-text-similarity', sagemaker_session=sagemaker_session, content_type='application/json', accept='application/json')

Now we will provide some sentence pairs in following format to compute similarity scores. Note that the model **DOES NOT** preprocess the sentences. So it is recommended you do so to get the best results.

In [4]:
data = {
  "instances": [
    {
      "data": {
        "first_sentence": "I want to buy a mobile phone",
        "second_sentence": "I wish to purchase a phone"
      },
      "instance_id": 10
    },
    {
      "data": {
        "first_sentence": "I want to buy a mobile phone",
        "second_sentence": "need to sell a car"
      },
      "instance_id": 2
    },
    {
      "data": {
        "first_sentence": "I want to buy a mobile phone",
        "second_sentence": "Give me your phone"
      },
      "instance_id": 3
    },
    {
      "data": {
        "first_sentence": "My wifi is not working",
        "second_sentence": "My internet goes down"
      },
      "instance_id": "a"
    },
    {
      "data": {
        "first_sentence": "My wifi is not working",
        "second_sentence": "My car wont start"
      },
      "instance_id": "b"
    },
    {
      "data": {
        "first_sentence": "My wifi is not working",
        "second_sentence": "electricity cut at my home"
      },
      "instance_id": "c"
    }
  ],
  "language": "en",
  "algorithm": "short-text-similarity"
}


data = json.dumps(data)

In [5]:
predictions = real_time_predictor.predict(data)
print(json.loads(predictions))

{'success': True, 'message': None, 'predictions': [{'instance_id': 10, 'score': 0.9518460631370544}, {'instance_id': 2, 'score': 0.585180401802063}, {'instance_id': 3, 'score': 0.6199266910552979}, {'instance_id': 'a', 'score': 0.8006324768066406}, {'instance_id': 'b', 'score': 0.5569400191307068}, {'instance_id': 'c', 'score': 0.590872585773468}]}


#### Error cases

Error cases will raise a ModelError exception with some message telling you if there were any mistakes in the request data

##### Language not supported

In [6]:
data = {
  "instances": [
    {
      "data": {
        "first_sentence": "I want to buy a mobile phone",
        "second_sentence": "I wish to purchase a phone"
      },
      "instance_id": 10
    }
  ],
  "language": "es",
  "algorithm": "short-text-similarity"
}

data = json.dumps(data)

try:
    predictions = real_time_predictor.predict(data)
    print(json.loads(predictions))
except real_time_predictor.sagemaker_session.sagemaker_runtime_client.exceptions.ModelError as e:
    print(e.args[0][:-40])

An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{"success": false, "message": "This predictor does not currently support the requested language"}". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/short-text-similarity in ac


##### Wrong/Missing keys for `first_sentence` or `second_sentence`

In [7]:
data = {
  "instances": [
    {
      "data": {
        "sentence1": "I want to buy a mobile phone",
        "second_sentence": "I wish to purchase a phone"
      },
      "instance_id": 1
    }
  ],
  "language": "en",
  "algorithm": "short-text-similarity"
}

data = json.dumps(data)

try:
    predictions = real_time_predictor.predict(data)
    print(json.loads(predictions))
except real_time_predictor.sagemaker_session.sagemaker_runtime_client.exceptions.ModelError as e:
    print(e.args[0][:-40])

An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{"success": false, "message": "Instance at index 0 is missing key first_sentence"}". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/short-text-similarity in ac


##### More than 1000 instances given

In [8]:
_instances = [{"data": {"first_sentence": "A", "second_sentence": "B"}} for _ in range(1005)]
data = {
    "instances": _instances,
    "language": "en",
    "algorithm": "short-text-similarity"
}

data = json.dumps(data)

try:
    predictions = real_time_predictor.predict(data)
    print(json.loads(predictions))
except real_time_predictor.sagemaker_session.sagemaker_runtime_client.exceptions.ModelError as e:
    print(e.args[0][:-40])

An error occurred (ModelError) when calling the InvokeEndpoint operation: Received client error (400) from model with message "{"success": false, "message": "This predictor supports only 1000 instances at a time"}". See https://us-east-2.console.aws.amazon.com/cloudwatch/home?region=us-east-2#logEventViewer:group=/aws/sagemaker/Endpoints/short-text-similarity in ac
